In [1]:
# For YOLO model
!pip install ultralytics -q

# For OCR model and data handling
!pip install tensorflow huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.7 MB/s eta 0:00:0000:0100:01


In [2]:
import os
import cv2
import yaml
import random
import tarfile
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from ultralytics import YOLO
import easyocr # Replaced keras_ocr

from huggingface_hub import hf_hub_download

print("Imports successful!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Imports successful!


In [3]:
def download_and_extract_dataset(repo_id, archive_filename, expected_internal_folder):
    """
    Downloads a .tar.gz file from Hugging Face Hub, extracts it, and returns the path
    to the main data folder inside.
    
    Args:
        repo_id (str): The Hugging Face repository ID (e.g., "username/repo-name").
        archive_filename (str): The name of the .tar.gz file in the repo.
        expected_internal_folder (str): The name of the top-level folder inside the archive.
        
    Returns:
        str: The local path to the extracted data folder.
    """
    # Define where to download and extract
    download_dir = "/content/downloads"
    extract_dir = "/content/extracted_datasets"
    os.makedirs(download_dir, exist_ok=True)
    os.makedirs(extract_dir, exist_ok=True)
    
    print(f"--- Processing {repo_id} ---")
    
    # 1. Download the file
    print(f"Downloading {archive_filename}...")
    downloaded_path = hf_hub_download(
        repo_id=repo_id,
        filename=archive_filename,
        repo_type="dataset",
        local_dir=download_dir
    )
    print(f"Downloaded to: {downloaded_path}")
    
    # 2. Extract the archive
    print(f"Extracting to {extract_dir}...")
    with tarfile.open(downloaded_path, "r:gz") as tar:
        tar.extractall(path=extract_dir)
    print("Extraction complete.")
    
    # 3. Return the path to the actual data folder
    dataset_path = os.path.join(extract_dir, expected_internal_folder)
    
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"Expected folder '{dataset_path}' not found after extraction.")
        
    print(f"✓ Dataset ready at: {dataset_path}\n")
    return dataset_path

In [4]:
# !!! IMPORTANT: Replace with your actual username !!!
HF_USERNAME = "zenitsu09"
YOLO_REPO_ID = f"{HF_USERNAME}/ccpd-yolo-detection"
YOLO_ARCHIVE_NAME = "yolo_dataset.tar.gz"
YOLO_INTERNAL_FOLDER = "ccpd_yolo" # This was the root folder you archived

yolo_dataset_path = download_and_extract_dataset(
    repo_id=YOLO_REPO_ID,
    archive_filename=YOLO_ARCHIVE_NAME,
    expected_internal_folder=YOLO_INTERNAL_FOLDER
)

# The path to the training configuration file
yolo_yaml_path = os.path.join(yolo_dataset_path, 'ccpd_yolo.yaml')
print(f"Path to YOLO config file: {yolo_yaml_path}")

--- Processing zenitsu09/ccpd-yolo-detection ---


yolo_dataset.tar.gz:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Downloaded to: /content/downloads/yolo_dataset.tar.gz
Extracting to /content/extracted_datasets...
Extraction complete.
✓ Dataset ready at: /content/extracted_datasets/ccpd_yolo

Path to YOLO config file: /content/extracted_datasets/ccpd_yolo/ccpd_yolo.yaml


In [5]:
import yaml

# Path to the YOLO dataset configuration file
yolo_yaml_path = '/content/extracted_datasets/ccpd_yolo/ccpd_yolo.yaml'

try:
    # Read the existing YAML content
    with open(yolo_yaml_path, 'r') as f:
        yolo_config = yaml.safe_load(f)

    # Update the 'path' to the correct extraction directory
    correct_dataset_path = '/content/extracted_datasets/ccpd_yolo'
    yolo_config['path'] = correct_dataset_path

    # Write the modified content back to the YAML file
    with open(yolo_yaml_path, 'w') as f:
        yaml.dump(yolo_config, f)

    print(f"Successfully updated the 'path' in {yolo_yaml_path} to: {correct_dataset_path}")

except FileNotFoundError:
    print(f"Error: The file {yolo_yaml_path} was not found.")
except Exception as e:
    print(f"An error occurred while modifying the file: {e}")

Successfully updated the 'path' in /content/extracted_datasets/ccpd_yolo/ccpd_yolo.yaml to: /content/extracted_datasets/ccpd_yolo


In [ ]:
# Load a pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')

print("Starting YOLOv8 training...")
# Train the model
results = model.train(
    data=yolo_yaml_path,
    epochs=3,
    imgsz=640,
    project='yolo_training',
    name='lp_detector_from_zip' # Changed name to avoid conflicts
)
print("YOLOv8 training complete!")

# Path to the best model
yolo_best_model_path = 'yolo_training/lp_detector_from_zip/weights/best.pt'
print(f"Best YOLO model saved at: {yolo_best_model_path}")

Starting YOLOv8 training...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/extracted_datasets/ccpd_yolo/ccpd_yolo.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=lp_detector_from_zip3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tru

train: Scanning /content/extracted_datasets/ccpd_yolo/labels/train.cache... 78767 images, 0 backgrounds, 0 corrupt: 100%|██████████| 78767/78767 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 478.1±271.8 MB/s, size: 95.6 KB)


val: Scanning /content/extracted_datasets/ccpd_yolo/labels/val.cache... 9846 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9846/9846 [00:00<?, ?it/s]


Plotting labels to yolo_training/lp_detector_from_zip3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to yolo_training/lp_detector_from_zip3
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3      3.53G      1.003      0.675      1.026         27        640: 100%|██████████| 4923/4923 [14:32<00:00,  5.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 308/308 [00:44<00:00,  6.86it/s]


                   all       9846       9846      0.994      0.997      0.994      0.768

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/3      3.64G     0.9441     0.4387      1.021         33        640: 100%|██████████| 4923/4923 [14:05<00:00,  5.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 308/308 [00:44<00:00,  6.93it/s]


                   all       9846       9846      0.995      0.998      0.994       0.79

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3      3.64G     0.8953      0.393      1.002         28        640:  74%|███████▍  | 3667/4923 [10:16<03:21,  6.23it/s]

In [ ]:
!ls /kaggle/working/yolo_training/lp_detector_from_zip